In [1]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

cudnn.benchmark = True

#set manual seed to a constant get a consistent output
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#loading the dataset
dataset = dset.CIFAR10(root="./data", download=True,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
nc=3

dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, num_workers=2)

#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# number of gpu's available
ngpu = 1
# input noise dimension
nz = 100
# number of generator filters
ngf = 64
#number of discriminator filters
ndf = 64

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output

netG = Generator(ngpu).to(device)
netG.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
print(netG)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
#load weights to test the model 
#netD.load_state_dict(torch.load('weights/netD_epoch_24.pth'))
print(netD)

criterion = nn.BCELoss()

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1.
fake_label = 0.

niter = 25
g_loss = []
d_loss = []

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch, niter, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        #save the output
        if i % 100 == 0:
            print('saving the output')
            vutils.save_image(real_cpu,'output/real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'output/fake_samples_epoch_%03d.png' % (epoch),normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'saved_model/cifar10_netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'saved_model/cifar10_netD_epoch_%d.pth' % (epoch))

Random Seed:  7007
Files already downloaded and verified
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2

[0/25][76/391] Loss_D: 0.4427 Loss_G: 7.8201 D(x): 0.9061 D(G(z)): 0.2534 / 0.0006
[0/25][77/391] Loss_D: 0.5207 Loss_G: 3.8718 D(x): 0.7363 D(G(z)): 0.0265 / 0.0256
[0/25][78/391] Loss_D: 0.6463 Loss_G: 10.6180 D(x): 0.9306 D(G(z)): 0.4113 / 0.0001
[0/25][79/391] Loss_D: 0.9210 Loss_G: 6.0178 D(x): 0.5508 D(G(z)): 0.0008 / 0.0064
[0/25][80/391] Loss_D: 0.1546 Loss_G: 4.1832 D(x): 0.9514 D(G(z)): 0.0805 / 0.0195
[0/25][81/391] Loss_D: 0.4757 Loss_G: 9.7341 D(x): 0.9626 D(G(z)): 0.3267 / 0.0002
[0/25][82/391] Loss_D: 0.7954 Loss_G: 5.7664 D(x): 0.5957 D(G(z)): 0.0019 / 0.0157
[0/25][83/391] Loss_D: 0.2276 Loss_G: 5.2449 D(x): 0.9430 D(G(z)): 0.1305 / 0.0077
[0/25][84/391] Loss_D: 0.1851 Loss_G: 5.7687 D(x): 0.9696 D(G(z)): 0.1339 / 0.0054
[0/25][85/391] Loss_D: 0.2397 Loss_G: 5.2185 D(x): 0.8744 D(G(z)): 0.0704 / 0.0078
[0/25][86/391] Loss_D: 0.3834 Loss_G: 4.2483 D(x): 0.8279 D(G(z)): 0.1176 / 0.0236
[0/25][87/391] Loss_D: 0.3911 Loss_G: 6.2504 D(x): 0.8925 D(G(z)): 0.1979 / 0.0028
[0/

[0/25][174/391] Loss_D: 0.2830 Loss_G: 7.2617 D(x): 0.8754 D(G(z)): 0.1139 / 0.0009
[0/25][175/391] Loss_D: 0.3076 Loss_G: 6.9832 D(x): 0.8528 D(G(z)): 0.1019 / 0.0013
[0/25][176/391] Loss_D: 0.2360 Loss_G: 8.0714 D(x): 0.9165 D(G(z)): 0.1178 / 0.0005
[0/25][177/391] Loss_D: 0.1340 Loss_G: 6.4760 D(x): 0.9117 D(G(z)): 0.0235 / 0.0021
[0/25][178/391] Loss_D: 0.1332 Loss_G: 6.1529 D(x): 0.9494 D(G(z)): 0.0728 / 0.0027
[0/25][179/391] Loss_D: 0.2022 Loss_G: 8.0394 D(x): 0.9470 D(G(z)): 0.1266 / 0.0004
[0/25][180/391] Loss_D: 0.1584 Loss_G: 6.3424 D(x): 0.8875 D(G(z)): 0.0146 / 0.0023
[0/25][181/391] Loss_D: 0.1506 Loss_G: 6.4928 D(x): 0.9536 D(G(z)): 0.0902 / 0.0020
[0/25][182/391] Loss_D: 0.1941 Loss_G: 6.0176 D(x): 0.9054 D(G(z)): 0.0541 / 0.0030
[0/25][183/391] Loss_D: 0.1716 Loss_G: 8.5432 D(x): 0.9743 D(G(z)): 0.1300 / 0.0002
[0/25][184/391] Loss_D: 0.2339 Loss_G: 6.5115 D(x): 0.8478 D(G(z)): 0.0101 / 0.0018
[0/25][185/391] Loss_D: 0.1616 Loss_G: 6.9765 D(x): 0.9657 D(G(z)): 0.1049 /

[0/25][274/391] Loss_D: 3.2962 Loss_G: 11.1729 D(x): 0.0959 D(G(z)): 0.0000 / 0.0003
[0/25][275/391] Loss_D: 0.1605 Loss_G: 5.9734 D(x): 0.8962 D(G(z)): 0.0066 / 0.0760
[0/25][276/391] Loss_D: 0.6759 Loss_G: 8.1827 D(x): 0.9767 D(G(z)): 0.2686 / 0.0013
[0/25][277/391] Loss_D: 0.3887 Loss_G: 4.1344 D(x): 0.7813 D(G(z)): 0.0354 / 0.0353
[0/25][278/391] Loss_D: 0.4093 Loss_G: 7.2102 D(x): 0.9626 D(G(z)): 0.2756 / 0.0029
[0/25][279/391] Loss_D: 0.2545 Loss_G: 5.5269 D(x): 0.8388 D(G(z)): 0.0390 / 0.0136
[0/25][280/391] Loss_D: 0.2220 Loss_G: 4.5529 D(x): 0.9188 D(G(z)): 0.0902 / 0.0223
[0/25][281/391] Loss_D: 0.2992 Loss_G: 6.9972 D(x): 0.9703 D(G(z)): 0.1956 / 0.0023
[0/25][282/391] Loss_D: 0.2208 Loss_G: 5.3392 D(x): 0.8594 D(G(z)): 0.0279 / 0.0082
[0/25][283/391] Loss_D: 0.2427 Loss_G: 3.8222 D(x): 0.8697 D(G(z)): 0.0782 / 0.0362
[0/25][284/391] Loss_D: 0.4693 Loss_G: 10.0332 D(x): 0.9486 D(G(z)): 0.3068 / 0.0001
[0/25][285/391] Loss_D: 1.5757 Loss_G: 1.6554 D(x): 0.3242 D(G(z)): 0.0008

[0/25][373/391] Loss_D: 0.6837 Loss_G: 7.6220 D(x): 0.9486 D(G(z)): 0.4102 / 0.0022
[0/25][374/391] Loss_D: 1.4150 Loss_G: 1.4006 D(x): 0.4018 D(G(z)): 0.0093 / 0.3157
[0/25][375/391] Loss_D: 0.7039 Loss_G: 8.1760 D(x): 0.9879 D(G(z)): 0.4623 / 0.0011
[0/25][376/391] Loss_D: 0.8061 Loss_G: 4.2143 D(x): 0.6091 D(G(z)): 0.0117 / 0.0347
[0/25][377/391] Loss_D: 0.3511 Loss_G: 4.3845 D(x): 0.9498 D(G(z)): 0.2099 / 0.0343
[0/25][378/391] Loss_D: 0.3113 Loss_G: 4.1448 D(x): 0.8680 D(G(z)): 0.1153 / 0.0253
[0/25][379/391] Loss_D: 0.3874 Loss_G: 3.6278 D(x): 0.8519 D(G(z)): 0.1490 / 0.0502
[0/25][380/391] Loss_D: 0.4120 Loss_G: 5.2794 D(x): 0.8825 D(G(z)): 0.2014 / 0.0099
[0/25][381/391] Loss_D: 0.4748 Loss_G: 2.1638 D(x): 0.7159 D(G(z)): 0.0298 / 0.2030
[0/25][382/391] Loss_D: 0.7986 Loss_G: 8.7403 D(x): 0.9884 D(G(z)): 0.4780 / 0.0007
[0/25][383/391] Loss_D: 1.6581 Loss_G: 4.3580 D(x): 0.3869 D(G(z)): 0.0054 / 0.0222
[0/25][384/391] Loss_D: 0.3088 Loss_G: 2.8573 D(x): 0.9304 D(G(z)): 0.1804 /

...

[23/25][341/391] Loss_D: 1.2412 Loss_G: 1.0104 D(x): 0.3771 D(G(z)): 0.0748 / 0.4143
[23/25][342/391] Loss_D: 0.8532 Loss_G: 3.2514 D(x): 0.9363 D(G(z)): 0.5043 / 0.0514
[23/25][343/391] Loss_D: 0.4684 Loss_G: 2.7728 D(x): 0.7480 D(G(z)): 0.1303 / 0.0818
[23/25][344/391] Loss_D: 0.3800 Loss_G: 2.6355 D(x): 0.8595 D(G(z)): 0.1847 / 0.0904
[23/25][345/391] Loss_D: 0.3207 Loss_G: 2.6919 D(x): 0.8710 D(G(z)): 0.1495 / 0.0888
[23/25][346/391] Loss_D: 0.3801 Loss_G: 3.0849 D(x): 0.8852 D(G(z)): 0.2049 / 0.0607
[23/25][347/391] Loss_D: 0.2471 Loss_G: 2.9266 D(x): 0.8730 D(G(z)): 0.0968 / 0.0697
[23/25][348/391] Loss_D: 0.3331 Loss_G: 3.8604 D(x): 0.9533 D(G(z)): 0.2315 / 0.0285
[23/25][349/391] Loss_D: 0.1746 Loss_G: 3.9045 D(x): 0.8909 D(G(z)): 0.0495 / 0.0305
[23/25][350/391] Loss_D: 0.2016 Loss_G: 3.4132 D(x): 0.8624 D(G(z)): 0.0396 / 0.0473
[23/25][351/391] Loss_D: 0.4116 Loss_G: 3.7670 D(x): 0.9365 D(G(z)): 0.2608 / 0.0340
[23/25][352/391] Loss_D: 0.1570 Loss_G: 4.6794 D(x): 0.9101 D(G(z

[24/25][49/391] Loss_D: 0.3358 Loss_G: 2.5021 D(x): 0.8348 D(G(z)): 0.1276 / 0.1049
[24/25][50/391] Loss_D: 0.4224 Loss_G: 2.9965 D(x): 0.8869 D(G(z)): 0.2441 / 0.0637
[24/25][51/391] Loss_D: 0.5542 Loss_G: 1.8895 D(x): 0.7065 D(G(z)): 0.1472 / 0.1896
[24/25][52/391] Loss_D: 0.3488 Loss_G: 3.0008 D(x): 0.9271 D(G(z)): 0.2226 / 0.0640
[24/25][53/391] Loss_D: 0.3390 Loss_G: 2.6054 D(x): 0.8253 D(G(z)): 0.1203 / 0.0956
[24/25][54/391] Loss_D: 0.2794 Loss_G: 3.0756 D(x): 0.9221 D(G(z)): 0.1693 / 0.0601
[24/25][55/391] Loss_D: 0.2014 Loss_G: 3.4420 D(x): 0.9298 D(G(z)): 0.1127 / 0.0434
[24/25][56/391] Loss_D: 0.2608 Loss_G: 2.4544 D(x): 0.8493 D(G(z)): 0.0800 / 0.1102
[24/25][57/391] Loss_D: 0.1868 Loss_G: 3.5112 D(x): 0.9741 D(G(z)): 0.1424 / 0.0396
[24/25][58/391] Loss_D: 0.1289 Loss_G: 3.8999 D(x): 0.9612 D(G(z)): 0.0827 / 0.0255
[24/25][59/391] Loss_D: 0.1322 Loss_G: 3.7130 D(x): 0.9374 D(G(z)): 0.0612 / 0.0338
[24/25][60/391] Loss_D: 0.1134 Loss_G: 3.4908 D(x): 0.9454 D(G(z)): 0.0536 /

[24/25][149/391] Loss_D: 0.2119 Loss_G: 2.2990 D(x): 0.8665 D(G(z)): 0.0533 / 0.1279
[24/25][150/391] Loss_D: 0.1512 Loss_G: 3.9120 D(x): 0.9867 D(G(z)): 0.1245 / 0.0271
[24/25][151/391] Loss_D: 0.2303 Loss_G: 5.4890 D(x): 0.9642 D(G(z)): 0.1654 / 0.0061
[24/25][152/391] Loss_D: 0.9251 Loss_G: 1.8607 D(x): 0.4617 D(G(z)): 0.0194 / 0.1920
[24/25][153/391] Loss_D: 0.7715 Loss_G: 1.2284 D(x): 0.7289 D(G(z)): 0.3183 / 0.3321
[24/25][154/391] Loss_D: 1.2298 Loss_G: 5.8741 D(x): 0.8065 D(G(z)): 0.5884 / 0.0042
[24/25][155/391] Loss_D: 3.1874 Loss_G: 0.1023 D(x): 0.0569 D(G(z)): 0.0205 / 0.9068
[24/25][156/391] Loss_D: 2.6781 Loss_G: 5.4861 D(x): 0.9935 D(G(z)): 0.8899 / 0.0102
[24/25][157/391] Loss_D: 1.9467 Loss_G: 1.1260 D(x): 0.2018 D(G(z)): 0.0190 / 0.3967
[24/25][158/391] Loss_D: 0.9314 Loss_G: 2.2336 D(x): 0.9213 D(G(z)): 0.5125 / 0.1412
[24/25][159/391] Loss_D: 0.5063 Loss_G: 3.0996 D(x): 0.8179 D(G(z)): 0.2376 / 0.0619
[24/25][160/391] Loss_D: 0.7501 Loss_G: 1.4774 D(x): 0.6055 D(G(z

[24/25][249/391] Loss_D: 0.0265 Loss_G: 7.1750 D(x): 0.9754 D(G(z)): 0.0013 / 0.0012
[24/25][250/391] Loss_D: 0.0591 Loss_G: 4.6117 D(x): 0.9767 D(G(z)): 0.0334 / 0.0159
[24/25][251/391] Loss_D: 0.0251 Loss_G: 6.3488 D(x): 0.9783 D(G(z)): 0.0020 / 0.0026
[24/25][252/391] Loss_D: 0.0187 Loss_G: 5.1772 D(x): 0.9897 D(G(z)): 0.0081 / 0.0083
[24/25][253/391] Loss_D: 0.0102 Loss_G: 6.8172 D(x): 0.9914 D(G(z)): 0.0015 / 0.0017
[24/25][254/391] Loss_D: 0.1491 Loss_G: 6.0491 D(x): 0.9935 D(G(z)): 0.1274 / 0.0032
[24/25][255/391] Loss_D: 0.2146 Loss_G: 7.4237 D(x): 0.8255 D(G(z)): 0.0006 / 0.0009
[24/25][256/391] Loss_D: 0.8319 Loss_G: 5.6762 D(x): 0.4758 D(G(z)): 0.0019 / 0.0045
[24/25][257/391] Loss_D: 4.1634 Loss_G: 0.0011 D(x): 0.0212 D(G(z)): 0.0085 / 0.9989
[24/25][258/391] Loss_D: 7.6254 Loss_G: 4.0485 D(x): 0.9989 D(G(z)): 0.9993 / 0.0356
[24/25][259/391] Loss_D: 1.3174 Loss_G: 1.6129 D(x): 0.3522 D(G(z)): 0.0844 / 0.2721
[24/25][260/391] Loss_D: 0.7246 Loss_G: 1.5650 D(x): 0.7497 D(G(z

[24/25][349/391] Loss_D: 0.8896 Loss_G: 5.3720 D(x): 0.9945 D(G(z)): 0.5282 / 0.0068
[24/25][350/391] Loss_D: 1.8048 Loss_G: 0.3715 D(x): 0.2102 D(G(z)): 0.0224 / 0.7100
[24/25][351/391] Loss_D: 1.6696 Loss_G: 4.0277 D(x): 0.9770 D(G(z)): 0.7470 / 0.0287
[24/25][352/391] Loss_D: 0.7823 Loss_G: 2.1601 D(x): 0.5582 D(G(z)): 0.0939 / 0.1650
[24/25][353/391] Loss_D: 0.4854 Loss_G: 2.4579 D(x): 0.8698 D(G(z)): 0.2637 / 0.1076
[24/25][354/391] Loss_D: 0.4864 Loss_G: 2.4339 D(x): 0.7926 D(G(z)): 0.1924 / 0.1203
[24/25][355/391] Loss_D: 0.5681 Loss_G: 2.8794 D(x): 0.8275 D(G(z)): 0.2730 / 0.0772
[24/25][356/391] Loss_D: 0.4911 Loss_G: 1.9840 D(x): 0.7761 D(G(z)): 0.1766 / 0.1747
[24/25][357/391] Loss_D: 0.7455 Loss_G: 6.2256 D(x): 0.9513 D(G(z)): 0.4506 / 0.0030
[24/25][358/391] Loss_D: 2.4857 Loss_G: 0.2399 D(x): 0.1398 D(G(z)): 0.0117 / 0.7998
[24/25][359/391] Loss_D: 2.5805 Loss_G: 5.7355 D(x): 0.9969 D(G(z)): 0.8879 / 0.0069
[24/25][360/391] Loss_D: 2.4659 Loss_G: 1.4005 D(x): 0.1357 D(G(z